In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime as datetime
from datetime import datetime as dt
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.chrome.options import Options
import warnings
from bs4 import BeautifulSoup, SoupStrainer


warnings.filterwarnings("ignore", category=DeprecationWarning) 
dictCalander = {"January":"01" ,
"February":"02", 
"March":"03", 
"April":"04", 
"May":"05", 
"June":"06", 
"July":"07", 
"August":"08", 
"September":"09", 
"October":"10",
"November":"11",
"December":"12"}

In [2]:
def dataLoad():
    return pd.read_excel("./Location.xlsx").applymap(str.lower)

In [3]:
def cheapflightsearch(startlocation ,endlocation , startdate , enddate,dataframe):
    airline = "";
    price = "";
    if(not((isValidLocation(dataframe,startlocation)) & (isValidLocation(dataframe,endlocation)))):
        return ("Invalid Location" , "-1")
    elif(not ((isValidDateFormat(startdate)) & ((isValidDateFormat(enddate) | (enddate is None) | (enddate ==""))))):
        return ("Invalid Date" , "-1")
    return (airline , price)

In [4]:
def isValidLocation(dataframe , location):
    return (location.lower() in dataframe['Name'].unique()) | (location.lower() in dataframe['Code'].unique())

In [5]:
def isValidDateFormat(dateValue):
    format = "%Y-%m-%d"
    present = dt.now().strftime(format)
    present = datetime.datetime.strptime(present, format)
    if dateValue is None:
        return False
    else:
        try:
            past = datetime.datetime.strptime(dateValue, format)
            if past>present:
                return True
            else:
                return False
        except ValueError:
            return False

In [9]:
def markupDetails(element,dayCheck):
    rowValue=0
    colvalue=0
    flag = False
    soup = BeautifulSoup(element, "lxml")
    dayPicker =soup.find_all("div", {"class": "DayPicker-Week"})
    for i in dayPicker:
        a = len(i.find_all("div",{"class" : "DayPicker-Day--outside"}))
        dateInnercell =i.find_all("div",{"class" : "dateInnerCell"})
        for j in dateInnercell:
            textValue = j.find_all('p')[0].text
            if(textValue == dayCheck):
                colvalue=colvalue+1
                flag =True
            else:
                colvalue=colvalue+1
            if(flag):
                break
        if(flag):
            rowValue =rowValue+1
            break
    element = "/html/body/div/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[3]/div[{}]/div[{}]".format(rowValue,a+colvalue)
    return element

In [40]:
def extractCheperFlight(element):
    soup = BeautifulSoup(element, "lxml")
    airline = soup.find('p', attrs={'class' : 'airlineName'}).text
    flightCode= soup.find('p', attrs={'class' : 'fliCode'}).text
    price= soup.find('p', attrs={'class' : 'white-space-no-wrap'}).text
    return (airline , flightCode , price)

In [47]:
def callCheapFlightStatus(source,destination,startDate,endDate):
    dataframe = dataLoad()
    result = cheapflightsearch(source,destination,startDate,endDate,dataframe)
    if (result[0]==""):
        chrome_options = Options()
        chrome_options.add_experimental_option("detach", True)
        driver = webdriver.Chrome('./chromedriver', options=chrome_options)
        driver.get("https://www.makemytrip.com/")
        try:
            driver.maximize_window()
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html')))
            driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/label/input").click()
            time.sleep(3)
            sourceFiled= driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[1]/div/div/div/input")
            sourceFiled.clear()
            sourceFiled.send_keys(source)
            time.sleep(3)
            driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[1]/div/div/div/div/div[1]/ul/li[1]/div/div[1]").click()
            time.sleep(3)
            destinationPath=driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/input")
            destinationPath.clear()
            destinationPath.send_keys(destination)
            time.sleep(3)
            driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/div/div[1]/ul/li[1]/div/div[1]").click()
            time.sleep(3)
            currentmonth = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[1]/div").text
            pastmonth = dictCalander.get(currentmonth.split()[0])
            startMonth=startDate.split("-")[1]
            startDay=startDate.split("-")[2]
            while(pastmonth!=startMonth):
                driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[1]/span[2]").click()
                time.sleep(3)
                currentmonth = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[1]/div").text
                pastmonth = dictCalander.get(currentmonth.split()[0])
            time.sleep(3)
            element = driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[3]").get_attribute('outerHTML')
            clickablemonth = markupDetails(element , startDay.lstrip('0'))
            driver.find_element_by_xpath(clickablemonth).click()
            time.sleep(3)
            driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[2]/p/a").click()
            time.sleep(3)
            currentUrl = driver.current_url
            print(currentUrl)
            time.sleep(3)
            if len(driver.find_elements(By.XPATH,"/html/body/div/div/div[2]/div[2]/div[2]/div"))>0:
                driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div[2]/div/div/div[3]/button").click()
            
            cheappriceElement=driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/div/div/div[1]/div[1]").get_attribute('outerHTML')
            cheapFlight = extractCheperFlight(cheappriceElement)
            print(cheapFlight)
            
            driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div[2]/span/span").click()
            time.sleep(3)
            
            sortpriceElement=driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/div/div/div[1]/div[1]").get_attribute('outerHTML')
            sortFlight = extractCheperFlight(sortpriceElement)
            print(sortFlight)
            
            driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div[1]/span/span").click()
            time.sleep(3)
            
            firstestpriceElement=driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/div/div/div[1]/div[1]").get_attribute('outerHTML')
            firstFlight = extractCheperFlight(firstestpriceElement)
            print(firstFlight)
            
        except Exception as e:
            print(e.args[0])
        finally:
            #driver.close()
            print("Executed")
    else:
        print(result)

In [48]:
callCheapFlightStatus("pnq","ccu","2022-07-02","")

https://www.makemytrip.com/flight/search?itinerary=PNQ-CCU-02/07/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng
('Go First', 'G8 394', '₹ 7,937')
('Go First', 'G8 394', '₹ 7,937')
('IndiGo', '6E 122, 6E 664', '₹ 7,938')
Executed
